In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from numpy import percentile
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
import re # for regular expressions
import pandas as pd 
pd.set_option("display.max_colwidth", 200) 
import string
from sklearn import metrics 
from sklearn.metrics import mean_squared_error,mean_absolute_error, make_scorer,classification_report,confusion_matrix,accuracy_score,roc_auc_score,roc_curve
from sklearn.model_selection import train_test_split,cross_val_score,KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import nltk # for text manipulation
from nltk.stem.porter import *
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from tqdm import tqdm
import gensim
from sklearn.linear_model import LogisticRegression
from scipy import stats 
from sklearn.naive_bayes import BernoulliNB
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import xgboost as xgb
import warnings 
warnings.filterwarnings("ignore")

%matplotlib inline
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords 
from wordcloud import WordCloud, STOPWORDS

In [ ]:
train = pd.read_csv('/kaggle/input/covid-19-nlp-text-classification/Corona_NLP_train.csv')
test  = pd.read_csv('/kaggle/input/covid-19-nlp-text-classification/Corona_NLP_test.csv')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
train.columns

In [ ]:
test.columns

In [ ]:
test.value_counts()

In [ ]:
train.value_counts()

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
test.nunique()

In [ ]:
train.nunique()

In [ ]:
train['text'] = train.OriginalTweet

test['text'] = test.OriginalTweet

In [ ]:
train.head()

In [ ]:
test.head()

## Converting a Selected Column Categorical To Numerical data 

In [ ]:
def change_def(x):
    if x ==  "Extremely Positive":
        return "1"
    elif x == "Extremely Negative":
        return "-1"
    elif x == "Negative":
        return "-1"
    elif x ==  "Positive":
        return "1"
    else:
        return "0"

train['label'] = train['Sentiment'].apply(lambda x:change_def(x))
test ['label'] = test ['Sentiment'].apply(lambda x:change_def(x))


train.label.value_counts()

In [ ]:
train.head()

In [ ]:
test.head()

## Visualize the text data using wordcloud

In [ ]:
def wordCloud(sentiment):
    text = ",".join(
               review for review in train[train['Sentiment'] == sentiment].OriginalTweet 
        if 'COVID' not in review and 'https' not in review and 'Covid' not in review)

    wordcloud = WordCloud(max_words=200, colormap='Set2', background_color="magenta").generate(text)
    plt.figure(figsize=(15,10))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.figure(1,figsize=(12, 12))
    plt.title('Prevalent words in ' + sentiment + ' tweets', fontsize=19)
    plt.show()

In [ ]:
wordCloud("Extremely Negative")

In [ ]:
wordCloud("Negative")

In [ ]:
wordCloud("Positive")

In [ ]:
wordCloud("Extremely Positive")

In [ ]:
wordCloud("Neutral")

In [ ]:
plot = sns.countplot(x='Sentiment', data=train).set_xticklabels(labels=['Neutral', 'Positive', 'Extremely Negative', 'Negative','Extremely Positive'],rotation=20)

In [ ]:
polt1=sns.catplot("TweetAt", data=train, kind="count", height=8)

## Text Cleaning

### 1) Removing Punctuations, Numbers, and Special Characters

In [ ]:
# remove special characters, numbers, punctuations
train['TweetAt'] = train['TweetAt'].str.replace('[^a-zA-Z#]+',' ')
test['TweetAt'] = test['TweetAt'].str.replace('[^a-zA-Z#]+',' ')

In [ ]:
train.head()

In [ ]:
test.head()

### 2) Removing @user

In [ ]:

def remove_pattern(input_txt, pattern):    # write function for removing @user
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i,'',input_txt)
    return input_txt

In [ ]:
train['TweetAt'] = np.vectorize(remove_pattern)(train['OriginalTweet'], '@[\w]*')  # create new column with removed @user
test['TweetAt'] = np.vectorize(remove_pattern)(test['OriginalTweet'], '@[\w]*') 

In [ ]:
train.head()

In [ ]:
test.head()

### 3) Removed HTTP And URLS From OriginalTweet

In [ ]:
import re
train['OriginalTweet'] = train['OriginalTweet'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])
test['OriginalTweet'] = test['OriginalTweet'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])

In [ ]:
train.head()

In [ ]:
test.head()

### 4) Removing Short Words

In [ ]:
train['TweetAt'] = train['TweetAt'].apply(lambda x: ' '.join([w for w in x.split() if len(w) > 2]))
test['TweetAt'] = test['TweetAt'].apply(lambda x: ' '.join([w for w in x.split() if len(w) > 2]))

In [ ]:
train.head()

In [ ]:
test.head()

### 5) Lower Casing

In [ ]:
def lower(text):
    low_text= text.lower()
    return low_text
train['text'] = train['text'].apply(lambda x:lower(x))
test ['text'] = test ['text'].apply(lambda x:lower(x))

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
new_data = train[['TweetAt','Sentiment']]
new_data.head()

## Spitting Our Dataset into Training And Testing Dataset

In [ ]:
train,test = train_test_split(new_data,test_size = 0.2,random_state=0,stratify = new_data.Sentiment.values)
print("train shape : ", train.shape)
print("test shape : ", test.shape)

## Use Of Counter Vectorizer For Multi Class Classification

In [ ]:
stop = list(stopwords.words('english'))
vectorizer = CountVectorizer(decode_error = 'replace',stop_words = stop)

X_train = vectorizer.fit_transform(train.TweetAt.values)
X_test = vectorizer.transform(test.TweetAt.values)

y_train = train.Sentiment.values
y_test = test.Sentiment.values

print("X_train.shape : ", X_train.shape)
print("X_test.shape : ", X_test.shape)
print("y_train.shape : ", y_train.shape)
print("y_test.shape : ", y_test.shape)

## Random Forest Classifier

In [ ]:
rf_clf = RandomForestClassifier()

rf_clf.fit(X_train,y_train)

rf_prediction = rf_clf.predict(X_test)
rf_accuracy = accuracy_score(y_test,rf_prediction)
print("Training accuracy Score    : ",rf_clf.score(X_train,y_train))
print("Testing accuracy Score : ",rf_accuracy )
print(classification_report(rf_prediction,y_test))

## Logistic Regression(lr)

In [ ]:
logisticreg = LogisticRegression()

logisticreg.fit(X_train, y_train)

logisticreg_prediction = logisticreg.predict(X_test)
logisticreg_accuracy = accuracy_score(y_test,logisticreg_prediction)
print("Training accuracy Score    : ",logisticreg.score(X_train,y_train))
print("Testing accuracy Score : ",logisticreg_accuracy )
print(classification_report(logisticreg_prediction,y_test))